<a href="https://colab.research.google.com/github/arnicola/pneumonia-prediction-with-transfered-learning-/blob/master/pneumonia_detection_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

In [0]:
local_zip = '/content/drive/My Drive/PythonData/KaggleDataset_Pneumonia/train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/My Drive/PythonData/KaggleDataset_Pneumonia')
zip_ref.close()

local_zip = '/content/drive/My Drive/PythonData/KaggleDataset_Pneumonia/test.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/My Drive/PythonData/KaggleDataset_Pneumonia')
zip_ref.close()

In [0]:

#Preparing the image datasets: paths
train_path = "/content/drive/My Drive/PythonData/KaggleDataset_Pneumonia/train"
test_path = "/content/drive/My Drive/PythonData/KaggleDataset_Pneumonia/test"
#valid_path = "/content/drive/My Drive/PythonData/KaggleDataset_Pneumonia/valid"

train_path_health = os.path.join(train_path, 'NORMAL')
train_path_pneumn = os.path.join(train_path, 'PNEUMONIA')

test_path_health = os.path.join(test_path, 'NORMAL')
test_path_pneumn = os.path.join(test_path, 'PNEUMONIA')

print("Train path health: " + train_path_health)
print("Train path health: " + train_path_pneumn)
print("Test path health: " + test_path_health)
print("Test path health: " + test_path_pneumn)

In [0]:
#Preparing image dataset: properties

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

#Train generator
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        class_mode='binary') # Since we use binary_crossentropy loss, we need binary labels

#Test generator
test_generator = train_datagen.flow_from_directory(
        test_path,
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        class_mode='binary') # Since we use binary_crossentropy loss, we need binary labels

In [0]:

#Download the weights file of the Inception V3
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O '/content/drive/My Drive/PythonData/KaggleDataset_Pneumonia/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [0]:
#Loading the weight into the pretrained model
local_weights_file = '/content/drive/My Drive/PythonData/KaggleDataset_Pneumonia/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

In [0]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [0]:
last_layer = pre_trained_model.get_layer('mixed7')
print( 'last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

pre_trained_model.summary()

In [0]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

# Configure and compile the model
model = Model(pre_trained_model.input, x)
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['acc'])

In [0]:
#Training
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=2,
      validation_data=test_generator,
      validation_steps=50,
      verbose=2)